<a href="https://colab.research.google.com/github/aniishap/ds2002/blob/main/Data_Project_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [132]:
!pip install PyMySQL
!pip install mysql.connector
!pip install sqlalchemy
!pip install mysql-connector-python
!pip install pyMysql



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 23.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for mysql.connector: filename=mysql_connector-2.2.9-cp310-cp310-linux_x86_64.whl size=247953 sha256=ec486a5c86cf8a1aa4e8df7d812b20b8702f87d66ceff94780be8e45e698ed17
  Stored in directory: /root/.cache/pip/wheels/76/48/9b/da67ff1a18fe8e9d428f9b1a177716d4a7d363d2bbe83bf6cf
Successfully built mysql.connector
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.5/21.5 MB 29.6 MB/s eta 0:00:00


In [133]:
import os
import datetime
import pymongo
import pprint
import pandas as pd


In [137]:
import os
import numpy
import pandas as pd
from sqlalchemy import create_engine

#Set the host name here
#If you are running against a database locally it would be local host or 127.0.0.1
host_name = "database.ds2002.org"
#host_ip = "127.0.0.1"
port = "3306"

user_id = "euv7bw" #Use yours here
pwd = "euv7bw!"  #Use yours here
db_name = "sakila"

In [138]:
def get_dataframe(user_id, pwd, host_name, db_name, sql_query):
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    dframe = pd.read_sql(sql_query, connection);
    connection.close()

    return dframe


def set_dataframe(user_id, pwd, host_name, db_name, df, table_name, pk_column, db_operation):
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()

    if db_operation == "insert":
        df.to_sql(table_name, con=connection, index=False, if_exists='replace')
        sqlEngine.execute(f"ALTER TABLE {table_name} ADD PRIMARY KEY ({pk_column});")

    elif db_operation == "update":
        df.to_sql(table_name, con=connection, index=False, if_exists='append')

    connection.close()


In [140]:
import os
import pymysql
import warnings
warnings.filterwarnings('ignore')

conn = pymysql.connect(host=host_name, user=user_id, password=pwd, database=db_name)
cursor = conn.cursor()

try:
    cursor.execute('SELECT * FROM actor;')

    for row in cursor.fetchmany(size=5):
        print(row)

    cursor.close()

except:
    print ("Error: unable to fetch data")

conn.close()



(1, 'PENELOPE', 'GUINESS', datetime.datetime(2006, 2, 15, 4, 34, 33))
(2, 'NICK', 'WAHLBERG', datetime.datetime(2006, 2, 15, 4, 34, 33))
(3, 'ED', 'CHASE', datetime.datetime(2006, 2, 15, 4, 34, 33))
(4, 'JENNIFER', 'DAVIS', datetime.datetime(2006, 2, 15, 4, 34, 33))
(5, 'JOHNNY', 'LOLLOBRIGIDA', datetime.datetime(2006, 2, 15, 4, 34, 33))
